<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


<h1>Test Uniform, Default and He Initialization on MNIST Dataset with Relu Activation</h1>



<h3>Objective for this Notebook<h3>    
<h5> 1. Learn how to Define Several Neural Network, Criterion function, Optimizer.</h5>
<h5> 2. Test Uniform, Default and He Initialization </h5>   



<h2>Table of Contents</h2>
<p>In this lab, you will test the Uniform Initialization, Default Initialization and He Initialization on the MNIST dataset with Relu Activation</p>

<ul>
    <li><a href="#Model">Neural Network Module and Training Function</a></li>
    <li><a href="#Makeup_Data">Make Some Data</a></li>
    <li><a href="#Cost">Define Several Neural Network, Criterion function, Optimizer</a></li>
    <li><a href="#Train">Test Uniform, Default and He Initialization</a></li>
    <li><a href="#Result">Analyze Results</a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>

<hr>


<h2>Preparation</h2>


We'll need the following libraries:  


In [1]:
# Import the libraries we need to use in this lab

# Using the following line code to install the torchvision library
# !mamba install -y torchvision

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
import matplotlib.pylab as plt
import numpy as np

torch.manual_seed(0)

<!--Empty Space for separating topics-->


<h2 id="Model">Neural Network Module and Training Function</h2> 


Define the neural network module or class with He Initialization


In [2]:
# Define the class for neural network model with He Initialization

class Net_He(nn.Module):
    
    # Constructor
    def __init__(self, Layers):
        super(Net_He, self).__init__()
        self.hidden = nn.ModuleList()

        for input_size, output_size in zip(Layers, Layers[1:]):
            linear = nn.Linear(input_size, output_size)
            torch.nn.init.kaiming_uniform_(linear.weight, nonlinearity='relu')
            self.hidden.append(linear)

    # Prediction
    def forward(self, x):
        L = len(self.hidden)
        for (l, linear_transform) in zip(range(L), self.hidden):
            if l < L - 1:
                x = F.relu(linear_transform(x))
            else:
                x = linear_transform(x)
        return x

Define the class or neural network with Uniform Initialization


In [3]:
# Define the class for neural network model with Uniform Initialization

class Net_Uniform(nn.Module):
    
    # Constructor
    def __init__(self, Layers):
        super(Net_Uniform, self).__init__()
        self.hidden = nn.ModuleList()

        for input_size, output_size in zip(Layers, Layers[1:]):
            linear = nn.Linear(input_size,output_size)
            linear.weight.data.uniform_(0, 1)
            self.hidden.append(linear)
    
    # Prediction
    def forward(self, x):
        L = len(self.hidden)
        for (l, linear_transform) in zip(range(L), self.hidden):
            if l < L - 1:
                x = F.relu(linear_transform(x))
            else:
                x = linear_transform(x)
                
        return x

Class or Neural Network with PyTorch Default Initialization


In [4]:
# Define the class for neural network model with PyTorch Default Initialization

class Net(nn.Module):
    
    # Constructor
    def __init__(self, Layers):
        super(Net, self).__init__()
        self.hidden = nn.ModuleList()

        for input_size, output_size in zip(Layers, Layers[1:]):
            linear = nn.Linear(input_size, output_size)
            self.hidden.append(linear)
        
    def forward(self, x):
        L=len(self.hidden)
        for (l, linear_transform) in zip(range(L), self.hidden):
            if l < L - 1:
                x = F.relu(linear_transform(x))
            else:
                x = linear_transform(x)
                
        return x

Define a function to train the model, in this case the function returns a Python dictionary to store the training loss and accuracy on the validation data 


In [5]:
# Define function to  train model

def train(model, criterion, train_loader, validation_loader, optimizer, epochs = 100):
    i = 0
    loss_accuracy = {'training_loss': [], 'validation_accuracy': []}  
    
    #n_epochs
    for epoch in range(epochs):
        total = 0
        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            z = model(x.view(-1, 28 * 28))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            total += loss.item()
        loss_accuracy['training_loss'].append(total)
        
        train_correct = 0
        for x, y in train_loader:
            yhat = model(x.view(-1, 28 * 28))
            _, label = torch.max(yhat, 1)
            train_correct += (label == y).sum().item()
        train_accuracy = 100 * (train_correct / len(train_dataset))
        
        correct = 0
        for x, y in validation_loader:
            yhat = model(x.view(-1, 28 * 28))
            _, label = torch.max(yhat, 1)
            correct += (label == y).sum().item()
        accuracy = 100 * (correct / len(validation_dataset))
        loss_accuracy['validation_accuracy'].append(accuracy)
        print(f'epoch {epoch}: train loss: {total:.4f}, train accuracy: {train_accuracy:.4f}, validation accuracy: {accuracy:.4f}')
    
    return loss_accuracy

<!--Empty Space for separating topics-->


<h2 id="Makeup_Data">Make some Data</h2> 


Load the training dataset by setting the parameters <code>train </code> to <code>True</code> and convert it to a tensor  by placing a transform object int the argument <code>transform</code>


In [6]:
# Create the training dataset

train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

Load the testing dataset by setting the parameters train  <code>False</code> and convert it to a tensor  by placing a transform object int the argument <code>transform</code>


In [7]:
# Create the validation dataset

validation_dataset = dsets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

Create the training-data loader and the validation-data loader object 


In [8]:
# Create the data loader for training and validation

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=2000, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000, shuffle=False)

<!--Empty Space for separating topics-->


<h2 id="Cost">Define Neural Network, Criterion function, Optimizer and Train the Model</h2> 


Create the criterion function  


In [9]:
# Create the criterion function

criterion = nn.CrossEntropyLoss()

Create a list that contains layer size 


In [10]:
# Create the parameters

input_dim = 28 * 28
output_dim = 10
layers = [input_dim, 100, 200, 100, output_dim]

<!--Empty Space for separating topics-->


<h2 id="Train">Test PyTorch Default Initialization, Xavier Initialization and Uniform Initialization</h2> 


Train the network using PyTorch Default Initialization


In [11]:
# Train the model with the default initialization

model = Net(layers)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
training_results = train(model, criterion, train_loader,validation_loader, optimizer, epochs=30)

epoch 0: train loss: 69.1157, train accuracy: 9.0417, validation accuracy: 8.9200
epoch 1: train loss: 69.0573, train accuracy: 9.6667, validation accuracy: 9.6200
epoch 2: train loss: 68.9978, train accuracy: 13.1333, validation accuracy: 12.8900
epoch 3: train loss: 68.9361, train accuracy: 18.9117, validation accuracy: 18.8900
epoch 4: train loss: 68.8715, train accuracy: 24.2383, validation accuracy: 24.2800
epoch 5: train loss: 68.8034, train accuracy: 26.3933, validation accuracy: 26.2600
epoch 6: train loss: 68.7308, train accuracy: 26.5083, validation accuracy: 26.2700
epoch 7: train loss: 68.6528, train accuracy: 25.8217, validation accuracy: 25.9500
epoch 8: train loss: 68.5676, train accuracy: 25.1183, validation accuracy: 25.4300
epoch 9: train loss: 68.4736, train accuracy: 24.7583, validation accuracy: 25.2000
epoch 10: train loss: 68.3699, train accuracy: 27.4417, validation accuracy: 28.2400
epoch 11: train loss: 68.2557, train accuracy: 30.4750, validation accuracy: 31

Train the network using He Initialization function


In [12]:
# Train the model with the He initialization

model_He = Net_He(layers)
optimizer = torch.optim.SGD(model_He.parameters(), lr=learning_rate)
training_results_He = train(model_He, criterion, train_loader, validation_loader, optimizer, epochs=30)

epoch 0: train loss: 67.8108, train accuracy: 29.5367, validation accuracy: 29.6800
epoch 1: train loss: 62.1594, train accuracy: 45.8000, validation accuracy: 46.6200
epoch 2: train loss: 55.7441, train accuracy: 57.7400, validation accuracy: 58.9400
epoch 3: train loss: 48.2872, train accuracy: 65.7767, validation accuracy: 67.3800
epoch 4: train loss: 40.6485, train accuracy: 71.2217, validation accuracy: 72.6300
epoch 5: train loss: 33.9873, train accuracy: 75.5800, validation accuracy: 77.0300
epoch 6: train loss: 28.8311, train accuracy: 78.4400, validation accuracy: 79.8000
epoch 7: train loss: 25.0389, train accuracy: 80.6083, validation accuracy: 81.7200
epoch 8: train loss: 22.2566, train accuracy: 82.2900, validation accuracy: 83.3300
epoch 9: train loss: 20.1740, train accuracy: 83.3967, validation accuracy: 84.2600
epoch 10: train loss: 18.5677, train accuracy: 84.5500, validation accuracy: 85.2100
epoch 11: train loss: 17.2998, train accuracy: 85.4000, validation accuracy

Train the network using Uniform Initialization function


In [13]:
# Train the model with the Uniform initialization

model_Uniform = Net_Uniform(layers)
optimizer = torch.optim.SGD(model_Uniform.parameters(), lr=learning_rate)
training_results_Uniform = train(model_Uniform, criterion, train_loader, validation_loader, optimizer, epochs=30)

epoch 0: train loss: 1686476.8029, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 1: train loss: 69.1133, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 2: train loss: 69.1083, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 3: train loss: 69.1035, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 4: train loss: 69.0990, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 5: train loss: 69.0948, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 6: train loss: 69.0908, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 7: train loss: 69.0871, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 8: train loss: 69.0835, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 9: train loss: 69.0802, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 10: train loss: 69.0771, train accuracy: 9.9433, validation accuracy: 10.3200
epoch 11: train loss: 69.0741, train accuracy: 9.9433, validation accuracy: 10.32

<!--Empty Space for separating topics-->


<h2 id="Result">Analyze Results</h2> 


Compare the training loss for each activation 


In [14]:
# Plot the loss

plt.plot(training_results_He['training_loss'], label='He')
plt.plot(training_results['training_loss'], label='Default')
plt.plot(training_results_Uniform['training_loss'], label='Uniform')
plt.ylabel('loss')
plt.xlabel('iteration ') 
plt.title('training loss iterations')
plt.legend()

Compare the validation loss for each model  


In [15]:
# Plot the accuracy

plt.plot(training_results_He['validation_accuracy'], label='He')
plt.plot(training_results['validation_accuracy'], label='Default')
plt.plot(training_results_Uniform['validation_accuracy'], label='Uniform') 
plt.ylabel('validation accuracy')
plt.xlabel('epochs ')   
plt.legend()
plt.show()

<!--Empty Space for separating topics-->


<h2>About the Authors:</h2> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


Other contributors: <a href="https://www.linkedin.com/in/michelleccarey/">Michelle Carey</a>, <a href="www.linkedin.com/in/jiahui-mavis-zhou-a4537814a">Mavis Zhou</a>


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2020-09-23  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>
-->
## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>
